## 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

토픽 모델링은 문서의 집합에서 토픽을 찾아내는 프로세스를 말합니다. 이는 검색 엔진, 고객 민원 시스템 등과 같이 문서의 주제를 알아내는 일이 중요한 곳에서 사용됩니다. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)은 토픽 모델링의 대표적인 알고리즘

문서1 : 저는 사과랑 바나나를 먹어요
문서2 : 우리는 귀여운 강아지가 좋아요
문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요
<각 문서의 토픽 분포>
문서1 : 토픽 A 100%
문서2 : 토픽 B 100%
문서3 : 토픽 B 60%, 토픽 A 40%
토픽A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%
토픽B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%

### 가정

DA는 문서의 집합으로부터 어떤 토픽이 존재하는지를 알아내기 위한 알고리즘입니다. LDA는 앞서 배운 빈도수 기반의 표현 방법인 BoW의 행렬 DTM 또는 TF-IDF 행렬을 입력으로 하는데, 이로부터 알 수 있는 사실은 LDA는 단어의 순서는 신경쓰지 않겠다는 겁니다.

LDA는 문서들로부터 토픽을 뽑아내기 위해서 이러한 가정을 염두해두고 있습니다. 모든 문서 하나, 하나가 작성될 때 그 문서의 작성자는 이러한 생각을 했습니다. '나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야.' 조금 더 구체적으로 알아보겠습니다. 각각의 문서는 다음과 같은 과정을 거쳐서 작성되었다고 가정합니다.

1) 문서에 사용할 단어의 개수 N을 정합니다.
    
    Ex) 5개의 단어를 정하였습니다.

2) 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.

    Ex) 위 예제와 같이 토픽이 2개라고 하였을 때 강아지 토픽을 60%, 과일 토픽을 40%와 같이 선택할 수 있습니다.

3) 문서에 사용할 각 단어를 (아래와 같이) 정합니다.

3-1) 토픽 분포에서 토픽 T를 확률적으로 고릅니다.

    Ex) 60% 확률로 강아지 토픽을 선택하고, 40% 확률로 과일 토픽을 선택할 수 있습니다.
    
3-2) 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.

    Ex) 강아지 토픽을 선택하였다면, 33% 확률로 강아지란 단어를 선택할 수 있습니다. 이제 3)을 반복하면서 문서를 완성합니다.
    
### 수행

사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.

-> 모든 단어를 K개중 하나의 토픽에 할당

-> 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행함

-> 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다.

### 잠재 디리클레 할당 VS 잠재 의미 분석 차이

LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.
LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.



In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
news_df['clean_doc'][1]
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

<ipython-input-2-49251654851e>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [4]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [8]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [9]:
print(dictionary[66])

faith


In [10]:
len(dictionary)

64281

In [14]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)


(0, '0.015*"government" + 0.010*"people" + 0.008*"would" + 0.008*"president"')
(1, '0.016*"people" + 0.015*"would" + 0.009*"think" + 0.007*"believe"')
(2, '0.023*"file" + 0.018*"output" + 0.017*"entry" + 0.015*"program"')
(3, '0.019*"picture" + 0.013*"sleeve" + 0.010*"allah" + 0.008*"cubs"')
(4, '0.076*"drive" + 0.045*"disk" + 0.043*"scsi" + 0.027*"hard"')
(5, '0.022*"filename" + 0.011*"borland" + 0.007*"postscript" + 0.007*"cica"')
(6, '0.014*"neutral" + 0.011*"ground" + 0.011*"unit" + 0.010*"cross"')
(7, '0.050*"bike" + 0.017*"riding" + 0.017*"ride" + 0.016*"motorcycle"')
(8, '0.007*"list" + 0.007*"group" + 0.007*"news" + 0.006*"american"')
(9, '0.013*"university" + 0.010*"april" + 0.009*"washington" + 0.008*"research"')
(10, '0.013*"said" + 0.013*"people" + 0.012*"armenian" + 0.011*"armenians"')
(11, '0.010*"health" + 0.008*"medical" + 0.007*"guns" + 0.006*"control"')
(12, '0.025*"game" + 0.025*"team" + 0.021*"games" + 0.018*"play"')
(13, '0.012*"jesus" + 0.007*"bible" + 0.007*"chri

In [19]:
print(ldamodel.print_topics())

[(0, '0.015*"government" + 0.010*"people" + 0.008*"would" + 0.008*"president" + 0.008*"encryption" + 0.008*"public" + 0.007*"security" + 0.007*"chip" + 0.006*"clipper" + 0.006*"keys"'), (1, '0.016*"people" + 0.015*"would" + 0.009*"think" + 0.007*"believe" + 0.007*"many" + 0.006*"even" + 0.006*"true" + 0.006*"know" + 0.005*"well" + 0.005*"question"'), (2, '0.023*"file" + 0.018*"output" + 0.017*"entry" + 0.015*"program" + 0.010*"window" + 0.010*"line" + 0.008*"section" + 0.008*"build" + 0.007*"check" + 0.007*"rules"'), (3, '0.019*"picture" + 0.013*"sleeve" + 0.010*"allah" + 0.008*"cubs" + 0.006*"suck" + 0.006*"filters" + 0.006*"weiss" + 0.006*"tigers" + 0.006*"countersteering" + 0.005*"babies"'), (4, '0.076*"drive" + 0.045*"disk" + 0.043*"scsi" + 0.027*"hard" + 0.023*"drives" + 0.021*"controller" + 0.017*"floppy" + 0.013*"tape" + 0.012*"bios" + 0.012*"disks"'), (5, '0.022*"filename" + 0.011*"borland" + 0.007*"postscript" + 0.007*"cica" + 0.007*"mydisplay" + 0.007*"paradox" + 0.006*"print

/home/sunny/sunny_study/20_08_20_after/nltk_study/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
import pyLDAvis
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
#pyLDAvis.display(vis)

/home/sunny/sunny_study/20_08_20_after/nltk_study/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


좌측의 원들은 각각의 20개의 토픽을 나타냅니다. 각 원과의 거리는 각 토픽들이 서로 얼마나 다른지를 보여줍니다. 만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미입니다. 위의 그림에서는 10번 토픽을 클릭하였고, 이에 따라 우측에는 10번 토픽에 대한 정보가 나타납니다. 한 가지 주의할 점은 LDA 모델의 출력 결과에서는 토픽 번호가 0부터 할당되어 0~19의 숫자가 사용된 것과는 달리 위의 LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는 이제 +1이 된 값인 1~20까지의 값을 가집니다.

In [16]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(1, 0.3201576), (10, 0.18424317), (17, 0.48188)]
1 번째 문서의 topic 비율은 [(1, 0.54046583), (9, 0.027483251), (18, 0.13264169), (19, 0.27833894)]
2 번째 문서의 topic 비율은 [(1, 0.34526646), (8, 0.018625583), (13, 0.16023782), (17, 0.21947329), (19, 0.24424613)]
3 번째 문서의 topic 비율은 [(0, 0.45418617), (4, 0.01743527), (10, 0.049847256), (13, 0.051189937), (16, 0.072318286), (17, 0.014569882), (19, 0.33023635)]
4 번째 문서의 topic 비율은 [(2, 0.102635674), (12, 0.26544172), (16, 0.19456369), (19, 0.40771732)]


/home/sunny/sunny_study/20_08_20_after/nltk_study/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

/home/sunny/sunny_study/20_08_20_after/nltk_study/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

/home/sunny/sunny_study/20_08_20_after/nltk_study/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,17.0,0.4819,"[(1, 0.32018417), (10, 0.18422507), (17, 0.481..."
1,1,1.0,0.5405,"[(1, 0.54045445), (9, 0.027483255), (18, 0.132..."
2,2,1.0,0.3452,"[(1, 0.34524542), (8, 0.018625814), (13, 0.160..."
3,3,0.0,0.4542,"[(0, 0.45418683), (4, 0.017435279), (10, 0.049..."
4,4,19.0,0.4077,"[(2, 0.10263851), (12, 0.26544234), (16, 0.194..."
5,5,13.0,0.4282,"[(1, 0.11432807), (8, 0.05966163), (13, 0.4281..."
6,6,14.0,0.5108,"[(1, 0.11193752), (2, 0.03425694), (6, 0.01509..."
7,7,1.0,0.6137,"[(1, 0.6137185), (2, 0.057010684), (4, 0.01772..."
8,8,10.0,0.4595,"[(10, 0.45952955), (15, 0.123934336), (16, 0.1..."
9,9,19.0,0.5742,"[(0, 0.039817248), (8, 0.06129415), (14, 0.120..."
